In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\capor\Downloads\dataset(in).csv")

In [3]:
df

,Unnamed: 0,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,318,34,Private,209297,HS-grad,9,Never-married,Machine-op-inspct,Not-in-family,Black,Male,0,2001,40,United-States,<=50K
1,11608,32,Private,107417,HS-grad,9,Never-married,Other-service,Not-in-family,White,Male,0,0,37,United-States,<=50K
2,2362,59,Local-gov,296253,Masters,14,Divorced,Prof-specialty,Unmarried,White,Female,8614,0,60,United-States,>50K
3,17387,44,Federal-gov,102238,HS-grad,9,Divorced,Craft-repair,Unmarried,White,Male,0,0,40,United-States,<=50K
4,22358,27,?,190650,Bachelors,13,Never-married,?,Unmarried,Asian-Pac-Islander,Male,0,0,25,Philippines,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29300,17852,21,Private,306114,Some-college,10,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
29301,20240,40,?,224361,9th,5,Divorced,?,Unmarried,White,Female,0,0,5,Cuba,<=50K
29302,1433,31,Private,251659,Some-college,10,Married-civ-spouse,Other-service,Husband,Asian-Pac-Islander,Male,0,1485,55,?,>50K
29303,25247,41,Private,39581,Some-college,10,Divorced,Adm-clerical,Not-in-family,Black,Female,0,0,24,El-Salvador,<=50K


In [4]:
df.drop(['Unnamed: 0','education'], axis=1, inplace=True)
import numpy as np

df.replace("?", np.nan, inplace=True)

In [5]:
df = df.dropna()

In [6]:
df

,age,workclass,fnlwgt,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,34,Private,209297,9,Never-married,Machine-op-inspct,Not-in-family,Black,Male,0,2001,40,United-States,<=50K
1,32,Private,107417,9,Never-married,Other-service,Not-in-family,White,Male,0,0,37,United-States,<=50K
2,59,Local-gov,296253,14,Divorced,Prof-specialty,Unmarried,White,Female,8614,0,60,United-States,>50K
3,44,Federal-gov,102238,9,Divorced,Craft-repair,Unmarried,White,Male,0,0,40,United-States,<=50K
5,43,Local-gov,163434,13,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,55,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29298,63,Private,294009,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,15024,0,45,United-States,>50K
29299,28,Private,319768,14,Married-civ-spouse,Sales,Husband,White,Male,0,0,45,France,>50K
29300,21,Private,306114,10,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
29303,41,Private,39581,10,Divorced,Adm-clerical,Not-in-family,Black,Female,0,0,24,El-Salvador,<=50K


In [12]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education.num', 'marital.status',
       'occupation', 'relationship', 'race', 'sex', 'capital.gain',
       'capital.loss', 'hours.per.week', 'native.country', 'income'],
      dtype='object')

In [17]:
df["age"].value_counts()

34    762
35    761
36    756
33    750
31    750
     ... 
84      8
82      7
83      4
85      3
88      3
Name: age, Length: 71, dtype: int64

In [8]:
df["education.num"].value_counts()

9     8838
10    6038
13    4555
14    1462
11    1179
7      949
12     907
6      723
4      491
15     489
5      409
16     339
8      337
3      267
2      138
1       39
Name: education.num, dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27160 entries, 0 to 29304
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             27160 non-null  int64 
 1   workclass       27160 non-null  object
 2   fnlwgt          27160 non-null  int64 
 3   education.num   27160 non-null  int64 
 4   marital.status  27160 non-null  object
 5   occupation      27160 non-null  object
 6   relationship    27160 non-null  object
 7   race            27160 non-null  object
 8   sex             27160 non-null  object
 9   capital.gain    27160 non-null  int64 
 10  capital.loss    27160 non-null  int64 
 11  hours.per.week  27160 non-null  int64 
 12  native.country  27160 non-null  object
 13  income          27160 non-null  object
dtypes: int64(6), object(8)
memory usage: 3.1+ MB


In [15]:
from sklearn.preprocessing import LabelEncoder

# Lista de columnas a codificar
columnas_categoricas = [
    "workclass", "marital.status", "occupation", "relationship",
    "race", "sex", "native.country", "income"
]

for col in columnas_categoricas:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])  # Transforma la columna# Guarda el encoder por si se necesita revertir después

# Ver los primeros registros transformados
print(df.head())

   age  workclass  fnlwgt  education.num  marital.status  occupation  \
0   34          2  209297              9               4           6   
1   32          2  107417              9               4           7   
2   59          1  296253             14               0           9   
3   44          0  102238              9               0           2   
5   43          1  163434             13               2          10   

   relationship  race  sex  capital.gain  capital.loss  hours.per.week  \
0             1     2    1             0          2001              40   
1             1     4    1             0             0              37   
2             4     4    0          8614             0              60   
3             4     4    1             0             0              40   
5             0     4    1             0             0              55   

   native.country  income  
0              38       0  
1              38       0  
2              38       1  
3         

C:\Users\capor\AppData\Local\Temp\ipykernel_23920\38784900.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])  # Transforma la columna# Guarda el encoder por si se necesita revertir después
C:\Users\capor\AppData\Local\Temp\ipykernel_23920\38784900.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])  # Transforma la columna# Guarda el encoder por si se necesita revertir después
C:\Users\capor\AppData\Local\Temp\ipykernel_23920\387849

In [16]:
df

,age,workclass,fnlwgt,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,34,2,209297,9,4,6,1,2,1,0,2001,40,38,0
1,32,2,107417,9,4,7,1,4,1,0,0,37,38,0
2,59,1,296253,14,0,9,4,4,0,8614,0,60,38,1
3,44,0,102238,9,0,2,4,4,1,0,0,40,38,0
5,43,1,163434,13,2,10,0,4,1,0,0,55,38,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29298,63,2,294009,13,2,9,0,4,1,15024,0,45,38,1
29299,28,2,319768,14,2,11,0,4,1,0,0,45,9,1
29300,21,2,306114,10,4,0,3,4,1,0,0,20,38,0
29303,41,2,39581,10,0,0,1,2,0,0,0,24,7,0


In [23]:
df["income"].value_counts()

0    20369
1     6791
Name: income, dtype: int64

## Modelo

In [40]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

def train_and_evaluate_models(df):
    # Definir X e y
    X = df.drop(columns=["income"])
    y = df["income"]

    # Dividir el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Escalar los datos
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Definir modelos e hiperparámetros
    models = {
        'Random Forest': RandomForestClassifier(random_state=0)
    }

    param_grid = {
        'Random Forest': {
            'n_estimators': [65, 70, 75, 100],  # Centrado alrededor de 70
            'max_depth': [6, 7],       # Centrado alrededor de 7
            'min_samples_split': [2, 3],   # Centrado alrededor de 2
            'min_samples_leaf': [1, 2, 3], # Centrado alrededor de 2
        }
    }

    # Resultados
    results = []
    
    # Realizar Grid Search para cada modelo
    for name, model in models.items():
        print(f"Training {name}...")
        grid_search = GridSearchCV(model, param_grid[name], cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train_scaled, y_train)

        # Obtener mejor modelo y parámetros
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_

        # Evaluar en conjunto de prueba
        y_pred = best_model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        # Registrar resultados
        results.append({
            'Model': name,
            'Best Parameters': best_params,
            'Train Accuracy (CV)': best_score,
            'Test Accuracy': accuracy,
            'Classification Report': report
        })

        print(f"{name} - Best Parameters: {best_params}")
        print(f"{name} - Train Accuracy (CV): {best_score}")
        print(f"{name} - Test Accuracy: {accuracy}\n")
        print(report)

    # Convertir resultados a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
results_df = train_and_evaluate_models(df)

Training Random Forest...
